In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
import pickle
import sys
from scipy.sparse import hstack
sys.path.insert(0,'/home/aniket/anaconda3/lib/python3.7/site-packages')
import gensim
from gensim.models import Word2Vec

In [3]:
rawData = pd.read_csv('/home/aniket/Music/data/ericsson/NLP_Data/train.csv') 

In [4]:
rawData.shape

(30336, 17)

In [5]:
textData = rawData[['summary','positives','negatives','advice_to_mgmt']]

In [6]:
rawData.fillna('NA',inplace=True)

In [7]:
sentences = textData.values.flatten()

In [11]:
sentences = [s for s in sentences if str(s)!='nan']

In [15]:
sentences = [nltk.word_tokenize(s) for s in sentences]

In [17]:
model = Word2Vec(sentences, min_count=1)

/home/aniket/anaconda3/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


KeyboardInterrupt: 

In [6]:
stop_words = set(stopwords.words('english')) 
vectorizer = CountVectorizer(stop_words=stop_words,ngram_range=(2,3))

In [7]:
enc = OneHotEncoder()

In [8]:
corpus = textData.apply(lambda x : ' '.join(x),axis=0)

In [9]:
corpus = corpus.str.cat(sep=' ')

In [10]:
vectorizer.fit([corpus])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 3), preprocessor=None,
        stop_words={"couldn't", 'a', 'what', 'of', 'some', 'such', 'be', 'the', 'mightn', 'my', 'up', 'wouldn', 'more', "haven't", 'was', 'all', 'not', 'had', 'how', "didn't", 'themselves', 'its', "you'll", 'from', 'by', 'needn', 'll', 'why', 'am', 'our', 'same', 'isn', 'and', 'hers', 'didn', "wasn't", 'we'...few', 'nor', 'most', 'or', "should've", 'himself', 'd', 'after', 'below', 'did', 'you', 're', 'don'},
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [11]:
textData.shape

(30336, 4)

filename='vectorizer.sav'
pickle.dump(vectorizer,open(filename,'wb'))

In [12]:
summary = vectorizer.transform(textData['summary'])
positives = vectorizer.transform(textData['positives'])
negatives = vectorizer.transform(textData['negatives'])
advice_to_mgmt = vectorizer.transform(textData['advice_to_mgmt'])

In [13]:
vecData = hstack((summary,positives,negatives,advice_to_mgmt))

In [14]:
vecData.shape

(30336, 6436760)

vecData = pd.DataFrame(vecData.todense(),columns=vectorizer.get_feature_names())

In [21]:
data = rawData.drop(columns=['summary','positives','negatives','advice_to_mgmt'])

In [22]:
data = data.drop(columns=['date','ID'])

In [23]:
encodedData = enc.fit_transform(data[['Place','location','status','job_title']])
encodedData = pd.DataFrame(encodedData.todense(),columns=enc.get_feature_names())

In [24]:
data = data.drop(columns=['Place','location','status','job_title'])

In [25]:
data = pd.concat([data, encodedData], axis=1, join_axes=[data.index])

In [27]:
data = hstack((data.to_sparse(),vecData))

TypeError: no supported conversion for types: (dtype('O'), dtype('int64'))

In [ ]:
textData.head(10)